In [9]:
import tweepy
import csv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# keys


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [7]:
Session = sessionmaker()
engine = create_engine('sqlite:///twitter.db')
Session.configure(bind=engine)

In [ ]:
hashtags = ['h1b']

for hashtag in hashtags:
    with open(hashtag+'.csv', 'r') as csvfile, open(hashtag+'-retweets.csv', 'a') as csvRetweetsFile:
        csvRetweetsWriter = csv.writer(csvRetweetsFile)
        spamreader = csv.reader(csvfile, delimiter=',')
        next(spamreader) # skip header
        for tweet in spamreader:
            if(len(tweet)>0):
                print("Looking for retweets of", tweet[0]) # tweet id
                retweets = [status for status in api.retweets(id=tweet[0])]
                if len(retweets)>0:
                    print("Found", len(retweets))
                    for retweet in retweets:
                        print("Retweet: ", retweet.id)
                        csvRetweetsWriter.writerow([retweet.id, retweet.retweeted_status.id, retweet.text.encode("utf-8"),
                        retweet.created_at, retweet.user.id])

In [52]:
csvfile.close()
csvRetweetsFile.close()

In [54]:
for hashtag in hashtags:
    tweetData = pd.read_csv(hashtag+'.csv', usecols=['tweet_id', 'retweeted', 'user_id', 'tweet_text','tweet_created_at','tweet_retweet_count','tweet_favorite_count'])
    userData = pd.read_csv(hashtag+'.csv', usecols=['user_id','user_name','user_location','user_friends_count'])
    placeData = pd.read_csv(hashtag+'.csv', usecols=['place_type','place_full_name','country'])

    retweetData = pd.read_csv(hashtag+'-retweets.csv', usecols=['retweet_id','source_tweet_id','tweet_text','tweet_created_at', 'user_id'])

    tweetData.to_sql('tweet', engine, if_exists='append')
    userData.to_sql('user', engine, if_exists='append')
    placeData.to_sql('place', engine, if_exists='append')
    retweetData.to_sql('retweet', engine, if_exists='append')

In [55]:
res1 = pd.read_sql_query('SELECT COUNT(*) FROM tweet', engine)
print("Number of tweets: ", res1)

Number of tweets:     COUNT(*)
0     13292


In [56]:
res2 = pd.read_sql_query('SELECT COUNT(DISTINCT user_id) FROM user', engine)
print("Number of users: ", res2)

Number of users:     COUNT(DISTINCT user_id)
0                      415


In [57]:
res3 = pd.read_sql_query('SELECT COUNT(DISTINCT place_full_name) FROM place', engine)
print("Number of locations: ", res3)

Number of locations:     COUNT(DISTINCT place_full_name)
0                                2


In [59]:
res4 = pd.read_sql_query('SELECT COUNT(DISTINCT retweet_id) FROM retweet', engine)
print("Number of retweets: ", res4)

Number of retweets:     COUNT(DISTINCT retweet_id)
0                          64


In [64]:
res4 = pd.read_sql_query('SELECT DISTINCT tweet.tweet_id, retweet.retweet_id FROM retweet \
LEFT OUTER JOIN tweet \
ON tweet.tweet_id = retweet.source_tweet_id', engine)
print(res4)

               tweet_id           retweet_id
0   1053783521561432065  1054238646235148289
1   1053783521561432065  1054034038309056512
2   1053783521561432065  1054017146391588866
3   1053768308317659137  1056081594895687680
4   1053768308317659137  1053891201907204096
5   1053768308317659137  1053824830716461056
6   1053748203617607680  1055615057047027713
7   1053748203617607680  1055613592043540480
8   1053748203617607680  1055578726568734725
9   1053748203617607680  1055562036934074379
10  1053748203617607680  1055545771792064512
11  1053748203617607680  1055543886498619392
12  1053748203617607680  1053832037591707648
13  1053748203617607680  1053758188430995456
14  1053726960038248448  1053832128641687553
15  1053722661019242496  1053841921540481024
16  1053719282281078784  1054190612789059585
17  1053719282281078784  1054099110360309760
18  1053719282281078784  1053896283247775744
19  1053719282281078784  1053894271303065600
20  1053719282281078784  1053880200088285184
21  105371